## Welcome to a qMRLab interactive blog post Jupyter Notebook!

If this is your first time running a Juptyer Notebook, there's a lot of tutorials available online to help. [Here's one](https://www.dataquest.io/blog/jupyter-notebook-tutorial/) for your convenience.

## Introduction

This notebook contains everything needed to reproduce the MP2RAGE T<sub>1</sub> blog post on the [qMRLab website](). In fact, this notebook generated the HTML for the blog post too! This notebook is currently running on a MyBinder server that only you can access, but if you want to be kept up-to-date on any changes that the developpers make to this notebook, you should go to it's [GitHub repository](https://github.com/qMRLab/t1_notebooks) and follow it by clicking the "Watch" button in the top right (you may need to create a GitHub account, if you don't have one already).

## Tips

Here's a few things you can do in this notebook

### Code
* Run the entire processing by clicking above on the "Kernel" tab, then "Restart & Run All". It will be complete when none of the cells have an asterix "\*" in the square brackets.
* To change the code, you need to click once on code cells. To re-run that cell, click the "Run" button above when the cell is selected.
  * **Note:** Cells can depend on previous cells, or even on previous runs of the cell itself, so it's best to run all the previous cells beforehand.
* This binder runs on SoS, which allows the mixing of Octave (i.e. an open-source MATLAB) and Python cells. Take a look a the drop down menu on the top right of the cells to know which one you are running.
* To transfer data from cells of one language to another, you need to create a new cell in the incoming language and run `%get (param name) --from (outgoing language)`. See cells below for several examples within this notebook.

### HTML
* To reproduce the HTML of the blog post, run the entire processing pipeline (see point one in the previous section), then save the notebook (save icon, top left). Now, click on the drop down menu on the left pannel, and select `%sossave --to html --force` . After a few seconds, it should output "Workflow saved to VariableFlipAngle.html" – click on the HTML name, and you're done!
* Cells with tags called "scratch" are not displayed in the generated HTML.
* Cells with the tag "report_output" display the output (e.g. figures) in the generated HTML.
* Currently in an un-run notebook, the HTML is not formatted like the website. To do so, run the Python module import cell (`# Module imports`) and then very last cell (`display(HTML(...)`).

**If you have any other questions or comments, please raise them in a [GitHub issue](https://github.com/qMRLab/t1_notebooks/issues).**

# Note

The following cell is meant to be displayed for instructional purposes in the blog post HTML when "All cells" gets displayed (i.e. the Octave code).

In [ ]:
% **Blog post code introduction**
% 
% Congrats on activating the "All cells" option in this interactive blog post =D
%
% Below, several new HTML blocks have appears prior to the figures, displaying the Octave/MATLAB code that was used to generate the figures in this blog post.
%
% If you want to reproduce the data on your own local computer, you simply need to have qMRLab installed in your Octave/MATLAB path and run the "startup.m" file, as is shown below.
%
% If you want to get under the hood and modify the code right now, you can do so in the Jupyter Notebook of this blog post hosted on MyBinder. The link to it is in the introduction above.

In [ ]:
# PYTHON CODE
# Module imports

import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

from IPython.core.display import display, HTML

<center><h1 style="font-family: timesnewroman;font-size: 40px;">MP2RAGE T<sub>1</sub> Mapping</h1></center>
<p>

<div class=blog_body>
<p style="text-align:justify;">
Dictionary-based MRI techniques capable of generating T<sub>1</sub> maps are increasing in popularity, due to their growing availability on clinical scanners, rapid scan times, and fast post-processing computation time, thus making quantitative T<sub>1</sub> mapping accessible for clinical applications. Generally speaking, dictionary-based quantitative MRI techniques use numerical dictionaries—databases of pre-calculated signal values simulated for a wide range of tissue and protocol combinations—during the image reconstruction or post-processing stages. Popular examples of dictionary-based techniques that have been applied to T<sub>1</sub> mapping are MR Fingerprinting (MRF) (Ma et al. 2013), certain flavours of compressed sensing (CS) (Doneva et al. 2010; Li et al. 2012), and Magnetization Prepared 2 Rapid Acquisition Gradient Echoes (MP2RAGE) (Marques et al. 2010). Dictionary-based techniques can usually be classified into one of two categories: techniques that use information redundancy from parametric data to assist in accelerated imaging (e.g. CS, MRF), or those that use dictionaries to estimate quantitative maps using the MR images after reconstruction. Because MP2RAGE is a technique implemented primarily for T<sub>1</sub> mapping, and it is becoming increasingly available as a standard pulse sequence on many MRI systems, the remainder of this section will focus solely on this technique. However, many concepts discussed are shared by other dictionary-based techniques.

</p>

<p style="text-align:justify;">
MP2RAGE is an extension of the conventional MPRAGE pulse sequence widely used in clinical studies (Haase et al. 1989; Mugler & Brookeman 1990). A simplified version of the MP2RAGE pulse sequence is shown in Figure 1. MP2RAGE can be seen as a hybrid between the inversion recovery and VFA pulse sequences: a 180° inversion pulse is used to prepare the magnetization for T<sub>1</sub> sensitivity at the beginning of each TRMP2RAGE, and then two images are acquired at different inversion times using gradient recalled echo (GRE) imaging blocks with low flip angles and short repetition times (TR). During a given GRE imaging block, each excitation pulse is followed by a constant in-plane (“y”) phase encode weighting (varied for each TRMP2RAGE), but with different 3D (“z”) phase encoding gradients (varied at each TR). The center of k-space for the 3D phase encoding direction is acquired at the TI for each GRE imaging block. The main motivation for developing the MP2RAGE pulse sequence was to provide a metric similar to MPRAGE, but with self-bias correction of the static (B<sub>0</sub>) and receive (B<sub>1</sub><sup>-</sup>) magnetic fields, and a first order correction of the transmit magnetic field (B<sub>1</sub><sup>+</sup>). However, because two images at different TIs are acquired (unlike MPRAGE, which only acquires data at a single TI), information about the T<sub>1</sub> values can also be inferred, thus making it possible to generate quantitative T<sub>1</sub> maps using this data.
</p>
</div>

<div class=figure_caption>
<p style="text-align:justify;">
<b>
Figure 1. Simplified diagram of an MP2RAGE pulse sequence. TR: repetition time between successive gradient echo readouts, TR<sub>MP2RAGE</sub>: repetition time between successive adiabatic 180° inversion pulses, TI<sub>1</sub> and TI<sub>2</sub>: inversion times, <i>θ<sub>1</sub></i> and <i>θ<sub>2</sub></i>: excitation flip angles. The imaging readout events occur within each TR using a constant in-plane phase encode (“y”) gradient set for each TR<sub>MP2RAGE</sub>, but varying 3D phase encode (“z”) gradients between each successive TR.
</b>
</p>
</div>

<p>
<center><img src="mp2rage_pulsesequence.png" style="width:800px;height:auto;"></center>

<center> <h2 style="font-family:timesnewroman;font-size:30px">Signal Modelling</h2> </center>

<div class=blog_body>
<p style="text-align:justify;">
Prior to considering the full signal equations, we will first introduce the equation for the MP2RAGE parameter (<i>S</i><sub>MP2RAGE</sub>) that is calculated in addition to the T<sub>1</sub> map. For complex data (magnitude and phase, or real and imaginary), the MP2RAGE signal (<i>S</i><sub>MP2RAGE</sub>) is calculated from the images acquired at two TIs (<i>S</i><sub>GRE,TI1</sub> and <i>S</i><sub>GRE,TI2</sub>) using the following expression (Marques et al. 2010):
</p>

<p style="text-align:justify;">
<center><img src="equation1.png" style="width:800px;height:auto;margin-bottom: 50px;margin-top: 50px;"></center>
</p>

<p style="text-align:justify;">
This value is bounded between [-0.5, 0.5], and helps reduce some B<sub>0</sub> inhomogeneity effects using the phase data. For real data, or magnitude data with polarity restoration, this metric is instead calculated as:
</p>

<p style="text-align:justify;">
<center><img src="equation2.png" style="width:800px;height:auto;margin-bottom: 50px;margin-top: 50px;"></center>
</p>

<p style="text-align:justify;">
Because MP2RAGE is a hybrid of pulse sequences used for inversion recovery and VFA, the resulting signal equations are more complex. Typically, a steady state is not achieved during the short train of GRE imaging blocks, so the signal at the center of k-space for each readout (which defines the contrast weighting) will depend on the number of phase-encoding steps. For simplicity, the equations presented here assume that the 3D phase-encoding dimension is fully sampled (no partial Fourier or parallel imaging acceleration). For this case (see appendix of (Marques et al. 2010) for derivation details), the signal equations are:
</p>

<p style="text-align:justify;">
<center><img src="equation3.png" style="width:800px;height:auto;margin-bottom: 50px;margin-top: 50px;"></center>
</p>

<p style="text-align:justify;">
<center><img src="equation4.png" style="width:800px;height:auto;margin-bottom: 50px;margin-top: 50px;"></center>
</p>

<p style="text-align:justify;">
where B<sub>1</sub><sup>-</sup> is the receive field sensitivity, “eff” is the adiabatic inversion pulse efficiency, ER = exp(-TR/T<sub>1</sub>), EA = exp(-TA/T<sub>1</sub>), EB = exp(-TB/T<sub>1</sub>), EC = exp(-TC/T<sub>1</sub>). The variables TA, TB, and TC are the three different delay times (TA: time between inversion pulse and beginning of the GRE<sub>1</sub> block, TB: time between the end of GRE<sub>1</sub> and beginning of GRE<sub>2</sub>, TC: time between the end of GRE<sub>2</sub> and the end of the TR). If no k-space acceleration is used (e.g. no partial Fourier or parallel imaging acceleration), then these values are TA = TI<sub>1</sub> - (n/2)TR, TB = TI<sub>2</sub> - (TI<sub>1</sub> + nTR), and TC = TR<sub>MP2RAGE</sub> - (TI<sub>2</sub> + (n/2)TR), where n is the number of voxels acquired in the 3D phase encode direction varied within each GRE block. The value m<sub>z,ss</sub> is the steady-state longitudinal magnetization prior to the inversion pulse, and is given by:
</p>

<p style="text-align:justify;">
<center><img src="equation5.png" style="width:800px;height:auto;margin-bottom: 50px;margin-top: 50px;"></center>
</p>

<p style="text-align:justify;">
<center><img src="equation6.png" style="width:800px;height:auto;margin-bottom: 50px;margin-top: 50px;"></center>
</p>

<p style="text-align:justify;">
    From Eqs. (3–6), it is evident that the MP2RAGE parameter <i>S</i><sub>MP2RAGE</sub> (Eqs. (1.11) and (1.12)) cancels out the effects of receive field sensitivity, T<sub>2</sub><sup>*</sup>, and M<sub>0</sub>. The signal sensitivity related to the transmit field (B<sub>1</sub><sup>+</sup>), hidden in Eqs. (3–6) within the flip angle values <i>θ<sub>1</sub></i> and <i>θ<sub>2</sub></i>, can also be reduced by careful pulse sequence protocol design (Marques et al. 2010), but not entirely eliminated (Marques & Gruetter 2013).
</p>


</div>

<center> <h2 style="font-family:timesnewroman;font-size:30px">Data Fitting</h2> </center>

<div class=blog_body>
<p style="text-align:justify;">
Dictionary-based techniques such as MP2RAGE do not typically use conventional minimization algorithms (e.g. Levenberg-Marquardt) to fit signal equations to observed data. Instead, the MP2RAGE technique uses pre-calculated signal values for a wide range of parameter values (e.g. T<sub>1</sub>), and then interpolation is done within this dictionary of values to estimate the T<sub>1</sub> value that matches the observed signal. This approach results in rapid post-processing times because the dictionaries can be simulated/generated prior to scanning and interpolating between these values is much faster than most fitting algorithms. This means that the quantitative image can be produced and displayed directly on the MRI scanner console rather than needing to be fitted offline.
</p>

</div>

<div class=figure_caption>
<p style="text-align:justify;">
<b>
    Figure 2. T<sub>1</sub> lookup table as a function of B<sub>1</sub> and <i>S</i><sub>MP2RAGE</sub> value. Inversion times used to acquire this magnitude image dataset were 800 ms and 2700 ms, the flip angles were 4° and 5° (respectively),  TR<sub>MP2RAGE</sub> = 6000 ms, and TR = 6.7 ms. The code that was used were shared open sourced by the authors of the original MP2RAGE paper (<a href="url">https://github.com/JosePMarques/MP2RAGE-related-scripts</a>).
</b>
</div>

In [ ]:
%% MATLAB/OCTAVE CODE
% Code used to generate the data required for Figure 5 of the blog post

clear all
cd ../qMRLab
startup


MP2RAGE.B0=7;           % in Tesla
MP2RAGE.TR=6;           % MP2RAGE TR in seconds 
MP2RAGE.TRFLASH=6.7e-3; % TR of the GRE readout
MP2RAGE.TIs=[800e-3 2700e-3];% inversion times - time between middle of refocusing pulse and excitatoin of the k-space center encoding
MP2RAGE.NZslices=[35 72];% Slices Per Slab * [PartialFourierInSlice-0.5  0.5]
MP2RAGE.FlipDegrees=[4 5];% Flip angle of the two readouts in degrees
invEFF=0.99;

B1_vector=0.005:0.05:1.9;
T1_vector=0.5:0.05:5.2;

npoints=40;


%% creates a lookup table of MP2RAGE intensities as a function of B1 and T1

k=0;
for b1val=B1_vector
    k=k+1;
    [Intensity T1vector ]=MP2RAGE_lookuptable(2,MP2RAGE.TR,MP2RAGE.TIs,b1val*MP2RAGE.FlipDegrees,MP2RAGE.NZslices,MP2RAGE.TRFLASH,'normal',invEFF);
    MP2RAGEmatrix(k,:)=interp1(T1vector,Intensity,T1_vector);
end;

%% make the matrix  MP2RAGEMatrix into T1_matrix(B1,ratio)
MP2RAGE_vector=linspace(-0.5,0.5,npoints);
k=0;
for b1val=B1_vector
    k=k+1;
    try

        T1matrix(k,:)=interp1(MP2RAGEmatrix(k,:),T1_vector,MP2RAGE_vector,'pchirp');
    catch
        temp=MP2RAGEmatrix(k,:);
        temp(isnan(temp))=linspace(-0.5,-1,length(find(isnan(temp)==1)));
        temp=interp1(temp,T1_vector,MP2RAGE_vector);

        T1matrix(k,:)=temp;
        
    end
end


In [ ]:
%get T1matrix --from Octave
%get T1matrix --from Octave
%get T1matrix --from Octave
%get T1matrix --from Octave
%get B1_vector --from Octave
%get MP2RAGE_vector --from Octave

In [ ]:
# PYTHON CODE

init_notebook_mode(connected=True)
# The polling here is to ensure that plotly.js has already been loaded before
# setting display alignment in order to avoid a race condition.

trace5 = go.Heatmap(x = MP2RAGE_vector,
                   y = B1_vector,
                   z=T1matrix,
                   zmin=0,
                   zmax=5,
                   colorscale='Portland',
                   xaxis='x2',
                   yaxis='y2',
                   visible=True,
                   name = 'T1 values (ms)')

data=[trace5]

layout = dict(
    width=560,
    height=345,
    margin = dict(
                t=40,
                r=50,
                b=50,
                l=80),
    annotations=[
         dict(
            x=-0.14,
            y=0.5,
            showarrow=False,
            text='B<sub>1</sub> values',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
          dict(
            x=0.5,
            y=-0.2,
            showarrow=False,
            text='S<sub>MP2RAGE</sub> values',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.5,
            y=1.15,
            showarrow=False,
            text='Lookup Table',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=1.17,
            y=1.15,
            showarrow=False,
            text='T<sub>1</sub> (ms)',
            font=dict(
                family='Times New Roman',
                size=20
            ),
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis = dict(range = [0,1], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 0.5]),
    yaxis = dict(range = [0,1], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1]),
    showlegend = False,
    autosize = False,
)


fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-heatmap', config = config)

<div class=blog_body>

<p style="text-align:justify;">
To produce T<sub>1</sub> maps with good accuracy and precision using dictionary-based interpolation methods, it is important that the signal curves are unique for each parameter value. MP2RAGE can produce good T<sub>1</sub> maps by using a dictionary with only dimensions (T<sub>1</sub>, <i>S</i><sub>MP2RAGE</sub>), since <i>S</i><sub>MP2RAGE</sub> is unique for each T<sub>1</sub> value for a given protocol  (Marques et al. 2010). However, as was noted above, <i>S</i><sub>MP2RAGE</sub> is also sensitive to B<sub>1</sub> because of <i>θ<sub>1</sub></i> and <i>θ<sub>2</sub></i> in Eqs. (1.13–1.16). The  B<sub>1</sub>-sensitivity can be reduced substantially with careful MP2RAGE protocol optimization (Marques et al. 2010), and further improved by including B<sub>1</sub> as one of the dictionary dimensions [T<sub>1</sub>, B<sub>1</sub>, <i>S</i><sub>MP2RAGE</sub>] (Figure 1.15).  This requires an additional acquisition of a B<sub>1</sub> map (Marques & Gruetter 2013), which lengthens the scan time. 
</p>

</div>

<div class=figure_caption>
<p style="text-align:justify;">
<b>
Figure 3. Example MP2RAGE dataset of a healthy adult brain at 7T and T<sub>1</sub> map. Inversion times used to acquire this magnitude image dataset were 800 ms and 2700 ms, the flip angles were 4° and 5° (respectively),  TR<sub>MP2RAGE</sub> = 6000 ms, and TR = 6.7 ms. The dataset and code that was used were shared open sourced by the authors of the original MP2RAGE paper (<a href="url">https://github.com/JosePMarques/MP2RAGE-related-scripts</a>).

</b>
</div>

In [ ]:
%% MATLAB/OCTAVE CODE
% Download variable flip angle brain MRI data for Figure 7 of the blog post

cmd = ['curl -L -o mp2rage.zip https://osf.io/8x2c9/download?version=1'];
[STATUS,MESSAGE] = unix(cmd);
unzip('mp2rage.zip');


In [ ]:
%% MATLAB/OCTAVE CODE
% Code used to generate the data required for Figure 5 of the blog post

clear all
cd ../qMRLab
startup

MP2RAGE.B0=7;           % in Tesla
MP2RAGE.TR=6;           % MP2RAGE TR in seconds 
MP2RAGE.TRFLASH=6.7e-3; % TR of the GRE readout
MP2RAGE.TIs=[800e-3 2700e-3];% inversion times - time between middle of refocusing pulse and excitatoin of the k-space center encoding
MP2RAGE.NZslices=[35 72];% Slices Per Slab * [PartialFourierInSlice-0.5  0.5]
MP2RAGE.FlipDegrees=[4 5];% Flip angle of the two readouts in degrees
MP2RAGE.filenameUNI='MP2RAGE_UNI.nii'; % file with UNI
MP2RAGE.filenameINV1='MP2RAGE_INV1.nii'; % file with UNI 
MP2RAGE.filenameINV2='MP2RAGE_INV2.nii'; % file with INV2 

% load the MP2RAGE data - it can be either the SIEMENS one scaled from
% 0 4095 or the standard -0.5 to 0.5
MP2RAGEimg=load_untouch_nii(MP2RAGE.filenameUNI, [], [], [], [], [], []);
MP2RAGEINV1img=load_untouch_nii(MP2RAGE.filenameINV1, [], [], [], [], [], []);
MP2RAGEINV2img=load_untouch_nii(MP2RAGE.filenameINV2, [], [], [], [], [], []);

[T1map , M0map , R1map]=T1M0estimateMP2RAGE(MP2RAGEimg,MP2RAGEINV2img,MP2RAGE,0.96);

% Code used to re-orient the images to make pretty figures, and to assign variables with the axis lengths.


T1_map = imrotate(squeeze(T1map.img(:,130,:)),180);
T1_map(T1map.img>5)=0;
T1_map = T1_map*1000; % Convert to ms

xAxis = [0:size(T1_map,2)-1];
yAxis = [0:size(T1_map,1)-1];

% Raw MRI data at different TI values
S_INV1 = imrotate(squeeze(MP2RAGEINV1img.img(:,130,:)),180);
S_INV2 = imrotate(squeeze(MP2RAGEINV2img.img(:,130,:)),180);
B1map = imrotate(-0.5+1/4095*double(squeeze(MP2RAGEimg.img(:,130,:))),180);


In [ ]:
%get T1_map --from Octave
%get S_INV1 --from Octave
%get S_INV2 --from Octave
%get B1map --from Octave
%get xAxis --from Octave
%get yAxis --from Octave

In [ ]:
init_notebook_mode(connected=True)
# The polling here is to ensure that plotly.js has already been loaded before
# setting display alignment in order to avoid a race condition.

trace1 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=S_INV1,
                   colorscale='Greys',
                   showscale = False,
                   visible=False,
                   name = 'Signal')
trace2 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=S_INV2,
                   colorscale='Greys',
                   showscale = False,
                   visible=False,
                   name = 'Signal')
trace3 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=B1map,
                   zmin=-0.5,
                   zmax=0.5,
                   colorscale='RdBu',
                   showscale = False,
                   visible=True,
                   name = 'S_MP2RAGE')
trace5 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=T1_map,
                   zmin=0.0,
                   zmax=5000,
                   colorscale='Portland',
                   xaxis='x2',
                   yaxis='y2',
                   visible=True,
                   name = 'T1 values (ms)')

data=[trace1, trace2, trace3, trace5]


updatemenus = list([
    dict(active=2,
         x = 0.09,
         xanchor = 'left',
         y = -0.15,
         yanchor = 'bottom',
         direction = 'up',
         font=dict(
                family='Times New Roman',
                size=16
            ),
         buttons=list([   
            dict(label = 'S<sub>INV1</sub>',
                 method = 'update',
                 args = [{'visible': [True, False, False, True]},
                         ]),
            dict(label = 'S<sub>INV2</sub>',
                 method = 'update',
                 args = [{'visible': [False, True, False, True]},
                           ]),
            dict(label = 'S<sub>MP2RAGE</sub>',
                 method = 'update',
                 args = [{'visible': [False, False, True, True]},
                           ])
        ])
    )
])

layout = dict(
    width=560,
    height=345,
    margin = dict(
                t=40,
                r=50,
                b=10,
                l=50),
    annotations=[
        dict(
            x=0.055,
            y=1.15,
            showarrow=False,
            text='Input Data',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.6,
            y=1.15,
            showarrow=False,
            text='T<sub>1</sub> map',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=1.22,
            y=1.15,
            showarrow=False,
            text='T<sub>1</sub> (ms)',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis = dict(range = [0,206], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 0.58]),
    yaxis = dict(range = [0,215], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1]),
    xaxis2 = dict(range = [0,206], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0.40, 0.98]),
    yaxis2 = dict(range = [0,215], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1], anchor='x2'),
    showlegend = False,
    autosize = False,
    updatemenus=updatemenus
)


fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-heatmap', config = config)

<div class=blog_body>

<p style="text-align:justify;">
The MP2RAGE pulse sequence is increasingly being distributed by MRI vendors, thus typically a data fitting package is also available to reconstruct the T<sub>1</sub> maps online. Alternatively, several open source packages to create T<sub>1</sub> maps from MP2RAGE data are available online (Marques 2017; de Hollander 2017), and for new users these are recommended—as opposed to programming one from scratch—as there are many potential pitfalls (e.g. adjusting the equations to handle partial Fourier or parallel imaging acceleration).
</p>

</div>

<center> <h2 style="font-family:timesnewroman;font-size:30px">Benefits and Pitfalls</h2> </center>

<div class=blog_body>
<p style="text-align:justify;">
This widespread availability and its turnkey acquisition/fitting procedures are a main contributing factor to the growing interest for including quantitative T<sub>1</sub> maps in clinical and neuroscience studies. T<sub>1</sub> values measured using MP2RAGE showed  high levels of reproducibility for the brain in an inter- and intra-site study at eight sites (same MRI hardware/software and at 7 T) of two subjects (Voelker et al. 2016). Not only does MP2RAGE have one of the fastest acquisition and post-processing times among quantitative T<sub>1</sub> mapping techniques, but it can accomplish this while acquiring very high resolution T<sub>1</sub> maps (1 mm isotropic at 3T and submillimeter at 7T, both in under 10 min (Fujimoto et al. 2014)), opening the doors to cortical studies which greatly benefit from the smaller voxel size (Waehnert et al. 2014; Beck et al. 2018; Haast et al. 2018).
</p>

<p style="text-align:justify;">
Despite these benefits, MP2RAGE and similar dictionary-based techniques have certain limitations that are important to consider before deciding to incorporate them in a study. Good reproducibility of the quantitative T<sub>1</sub> map is dependent on using one pre-calculated dictionary. If two different dictionaries are used (e.g. cross-site with different MRI vendors), the differences in the dictionary interpolations will likely result in minor differences in T<sub>1</sub> estimates for the same data. Also, although the B1-sensitivity of the MP2RAGE T<sub>1</sub> maps can be reduced with proper protocol optimization, it can be substantial enough that further correction using a measured B<sub>1</sub> map should be done (Marques & Gruetter 2013; Haast et al. 2018). However B<sub>1</sub> mapping brings an additional potential source of error, so carefully selecting a B<sub>1</sub> mapping technique and accompanying post-processing method (e.g. filtering) should be done before integrating it in a T<sub>1</sub> mapping protocol (Boudreau et al. 2017). Lastly, the MP2RAGE equations (and thus, dictionaries) assume monoexponential longitudinal relaxation, and this has been shown to result in suboptimal estimates of the long T<sub>1</sub> component for a biexponential relaxation model (Rioux et al. 2016), an effect that becomes more important at higher fields.
</p>
</div>

<center> <h2 style="font-family:timesnewroman;font-size:30px">Works Cited</h2> </center>

<div class=biblio_body>
<p style="text-align:justify;">
Beck, E.S. et al., 2018. Improved Visualization of Cortical Lesions in Multiple Sclerosis Using 7T MP2RAGE. <i>Am. J. Neuroradiol.</i> Available at: <a href="http://dx.doi.org/10.3174/ajnr.A5534">http://dx.doi.org/10.3174/ajnr.A5534</a>.
</p>

<p style="text-align:justify;">
Boudreau, M. et al., 2017. B<sub>1</sub> mapping for bias-correction in quantitative T<sub>1</sub> imaging of the brain at 3T using standard pulse sequences. <i>J. Magn. Reson. Imaging</i>, 46(6), pp.1673–1682.
</p>

<p style="text-align:justify;">
Doneva, M. et al., 2010. Compressed sensing reconstruction for magnetic resonance parameter mapping. <i>Magn. Reson. Med.</i>, 64(4), pp.1114–1120.
</p>

<p style="text-align:justify;">
Fujimoto, K. et al., 2014. Quantitative comparison of cortical surface reconstructions from MP2RAGE and multi-echo MPRAGE data at 3 and 7 T. <i>NeuroImage</i>, 90, pp.60–73.
</p>

<p style="text-align:justify;">
Haase, A. et al., 1989. Inversion recovery snapshot FLASH MR imaging. <i>J. Comput. Assist. Tomogr.</i>, 13(6), pp.1036–1040.
</p>

<p style="text-align:justify;">
Haast, R.A.M., Ivanov, D. & Uludağ, K., 2018. The impact of B<sub>1</sub><sup>+</sup> correction on MP2RAGE cortical T<sub>1</sub> and apparent cortical thickness at 7T. <i>Hum. Brain Mapp.</i>, 39(6), pp.2412–2425.
</p>

<p style="text-align:justify;">
de Hollander, G., 2017. PyMP2RAGE. Available at: <a href="https://github.com/Gilles86/pymp2rage">https://github.com/Gilles86/pymp2rage</a> [Accessed January 2, 2019].
</p>

<p style="text-align:justify;">
Li, W., Griswold, M. & Yu, X., 2012. Fast cardiac T<sub>1</sub> mapping in mice using a model-based compressed sensing method. <i>Magn. Reson. Med.</i>, 68(4), pp.1127–1134.
</p>

<p style="text-align:justify;">
Ma, D. et al., 2013. Magnetic resonance fingerprinting. <i>Nature</i>, 495(7440), pp.187–192.
</p>

<p style="text-align:justify;">
Marques, J., 2017. MP2RAGE related scripts. Available at: <a href="https://github.com/JosePMarques/MP2RAGE-related-scripts">https://github.com/JosePMarques/MP2RAGE-related-scripts</a> [Accessed January 2, 2019].
</p>

<p style="text-align:justify;">
Marques, J.P. et al., 2010. MP2RAGE, a self bias-field corrected sequence for improved segmentation and T<sub>1</sub>-mapping at high field. <i>NeuroImage</i>, 49(2), pp.1271–1281.
</p>

<p style="text-align:justify;">
Marques, J.P. & Gruetter, R., 2013. New Developments and Applications of the MP2RAGE Sequence - Focusing the Contrast and High Spatial Resolution R<sub>1</sub> Mapping. <i>PloS one</i>, 8(7), p.e69294.
</p>

<p style="text-align:justify;">
Mugler, J.P., 3rd & Brookeman, J.R., 1990. Three-dimensional magnetization-prepared rapid gradient-echo imaging (3D MP RAGE). <i>Magn. Reson. Med.</i>, 15(1), pp.152–157.
</p>

<p style="text-align:justify;">
Rioux, J.A., Levesque, I.R. & Rutt, B.K., 2016. Biexponential longitudinal relaxation in white matter: Characterization and impact on T<sub>1</sub> mapping with IR-FSE and MP2RAGE. <i>Magn. Reson. Med.</i>, 75(6), pp.2265–2277.
</p>

<p style="text-align:justify;">
Voelker, M.N. et al., 2016. The traveling heads: multicenter brain imaging at 7 Tesla. <i>Magma</i>, 29(3), pp.399–415.
</p>

<p style="text-align:justify;">
Waehnert, M.D. et al., 2014. Anatomically motivated modeling of cortical laminae. <i>NeuroImage</i>, 93 Pt 2, pp.210–220.
</p>

</div>

<center> <h2 style="font-family:timesnewroman;font-size:30px">Errata</h2> </center>

<div class=biblio_body>
<ol> 
<li><p style="text-align:justify;">
A previous version of this text incorrectly stated the equations for TB and TC. They were written as TB = TI<sub>2</sub> - TI<sub>1</sub> + (n/2)TR and TC = TR<sub>MP2RAGE</sub> - TI<sub>2</sub> + (n/2)TR, whereas they should have been TB = TI<sub>2</sub> - (TI<sub>1</sub> + nTR) and TC = TR<sub>MP2RAGE</sub> - (TI<sub>2</sub> + (n/2)TR). This has been corrected in the text. Thank you to Marc-Antoine Fortin (<a href="https://www.linkedin.com/in/marc-antoine-fortin-7b3777163/?originalSubdomain=ca" target="_blank">LinkedIn</a>, <a href="https://twitter.com/marcus_pulus" target="_blank">Twitter</a>) from the <a href="https://www.mcgill.ca/medphys/mri-methods-research-group" target="_blank">MRI Methods Research Group</a> at McGill University for notifying us of this error. 
<p style="text-align:justify;">
</li>
</ol>

<p style="text-align:justify;">
</div>

In [ ]:
# PYTHON CODE

display(HTML(
    '<style type="text/css">'
    '.output_subarea {'
        'display: block;'
        'margin-left: auto;'
        'margin-right: auto;'
    '}'
    '.blog_body {'
        'line-height: 2;'
        'font-family: timesnewroman;'
        'font-size: 18px;'
        'margin-left: 0px;'
        'margin-right: 0px;'
    '}'
    '.biblio_body {'
        'line-height: 1.5;'
        'font-family: timesnewroman;'
        'font-size: 18px;'
        'margin-left: 0px;'
        'margin-right: 0px;'
    '}'
    '.note_body {'
        'line-height: 1.25;'
        'font-family: timesnewroman;'
        'font-size: 18px;'
        'margin-left: 0px;'
        'margin-right: 0px;'
        'color: #696969'
    '}'
    '.figure_caption {'
        'line-height: 1.5;'
        'font-family: timesnewroman;'
        'font-size: 16px;'
        'margin-left: 0px;'
        'margin-right: 0px'
    '</style>'
))